In [1]:
REFERENCE = 'Reference'
PROJECT = 'Project'

%load_ext tensorboard

from model.model import *
from process_func import *
import os
DATA_DIR = f'datasets/'
map = {'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs', 'Major': 'Military', 
       'Col': 'Military', 'Dr' : 'Other', 'Rev' : 'Other', 'Capt': 'Military', 
       'Jonkheer': 'Royal', 'Sir': 'Royal', 'Lady': 'Royal', 
       'Don': 'Royal', 'Countess': 'Royal', 'Dona': 'Royal'}
titles =['Miss', 'Mr', 'Mrs', 'Royal', 'Other', 'Master', 'Military']

# Run for validation

In [2]:
data = Titanic(DATA_DIR, show_head = False)
prepath = 'preprocessed'
try:
    if not os.path.exists(prepath):
        os.makedirs(prepath)
except OSError:
    print('Error Creating Directory...')
data.Preprocess(map, titles, PROJECT)
data._data.to_csv(prepath + f"/encoded_{PROJECT}.csv", index = False)
dataset = data.GetXandY()

# Run for validation

Initializing...
Data Loaded.
Data Preprocessing...
Done Preprocessing.
Returned Data Dictionary


In [3]:
# ref_model = ReferenceModel(dataset['x_train'].shape)
# CP_dir_ref = SetCheckpoint(REFERENCE)
# TB_dir_ref = SetLog(REFERENCE)

In [4]:
proj_model = ProjectModel(dataset['x_train'].shape)
CP_dir_proj = SetCheckpoint(PROJECT)
TB_dir_proj = SetLog(PROJECT)
# Run for validation

Model: "ProjectModel"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 19)]              0         
                                                                 
 Dense0 (Dense)              (None, 16)                320       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 batch_normalization (BatchN  (None, 16)               64        
 ormalization)                                                   
                                                                 
 Dense1 (Dense)              (None, 16)                272       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                      

In [5]:
# BATCH_SIZE = 128
# EPOCHS = 100
# callbacks_ref = DefCallbacks(REFERENCE, CP = CP_dir_ref, TB = TB_dir_ref)
# history_ref = ref_model.fit(dataset['x_train'], dataset['y_train'], callbacks = callbacks_ref, 
#           validation_split = 0.2, batch_size=BATCH_SIZE, epochs = EPOCHS, verbose = 2)

In [6]:
BATCH_SIZE = 128
EPOCHS = 600
callbacks_proj = DefCallbacks(PROJECT, CP = CP_dir_proj, TB = TB_dir_proj)
history_proj = proj_model.fit(dataset['x_train'], dataset['y_train'], callbacks = callbacks_proj, 
          validation_split = 0.2, batch_size=BATCH_SIZE, epochs = EPOCHS, verbose = 0)

In [ ]:
# submission = PerformanceCheck(ref_model, CP_dir_ref, dataset['x_test'], DATA_DIR)
# submission.to_csv(f"submission/submission_{REFERENCE}.csv", index = False)
submission = PerformanceCheck(proj_model, CP_dir_proj, dataset['x_test'], DATA_DIR)
submission.to_csv(f"submission/submission_{PROJECT}.csv", index = False)

3/3 [==============================] - 0s 3ms/step
0    0.060042
1    0.158263
2    0.080631
3    0.083024
4    0.928435
5    0.065761
6    0.640785
7    0.134921
8    0.255343
9    0.083024
dtype: float32
ProjectModel_Accuracy :  85.89 %
